In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from IPython.core.interactiveshell import InteractiveShell
# import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
# 配置项
# # 这个要放到设置中文之前否则还是小方框
# plt.style.use("seaborn")

# # 指定默认字体 用来正常显示中文标签
# plt.rcParams['font.sans-serif'] = ['SimHei']
# # 解决保存图像是负号'-'显示为方块的问题
# plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

In [4]:
def preprocess_data(x, y):
    # 这里的预处理方式 对结果有影响
#     x = tf.cast(x, dtype=tf.float32) / 255 # accuracy: 0.4613
#     x = tf.cast(x, dtype=tf.float32) / 255 -0.5 # loss: 1.6729 - accuracy: 0.4959
    x = tf.cast(x, dtype=tf.float32) / 125.5 - 1 # loss: 1.6406 - accuracy: 0.4902


    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [6]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [7]:
y_train = tf.squeeze(y_train, axis=1)
y_test = tf.squeeze(y_test, axis=1)


# 因为是10分类 所以depth是10
y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, depth=10)


In [8]:
y_train.shape, y_test.shape

(TensorShape([50000, 10]), TensorShape([10000, 10]))

In [9]:
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

batch_size = 128
db_train = db_train.map(preprocess_data).shuffle(10000).batch(batch_size)
db_test = db_test.map(preprocess_data).batch(batch_size)

In [10]:
sample = next(iter(db_train))
sample[0].shape, sample[1].shape

(TensorShape([128, 32, 32, 3]), TensorShape([128, 10]))

In [11]:
# 自定义layer和模型
class MyDense(keras.layers.Layer):
    def __init__(self, input_dim, out_dim):
        super(MyDense, self).__init__()
        
        self.kernel = self.add_variable('w', [input_dim, out_dim])
        
    def call(self, input_data, training=None):
        x = input_data @ self.kernel
        return x
    
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.dense1 = MyDense(32*32*3, 256)
        self.dense2 = MyDense(256, 128)
        self.dense3 = MyDense(128, 64)
        self.dense4 = MyDense(64, 32)
        self.dense5 = MyDense(32, 10)
        
    def call(self, input_data, training=None):
        x = tf.reshape(input_data, [-1, 32 * 32 * 3])
        x = self.dense1(x)
        x = tf.nn.relu(x)
        
        x = self.dense2(x)
        x = tf.nn.relu(x)
        
        x = self.dense3(x)
        x = tf.nn.relu(x)
        
        x = self.dense4(x)
        x = tf.nn.relu(x)
        
        x = self.dense5(x)
        
        return x
        
        

In [12]:
model = MyModel()

model.compile(optimizer=tf.optimizers.Adam(1e-3), 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy']
             )


In [13]:
model.fit(db_train, epochs=10, validation_data=db_test)

Epoch 1/10
391/391 [==============================] - 8s 21ms/step - loss: 1.7191 - accuracy: 0.3434 - val_loss: 1.5613 - val_accuracy: 0.4506
Epoch 2/10
391/391 [==============================] - 7s 18ms/step - loss: 1.5025 - accuracy: 0.4619 - val_loss: 1.4805 - val_accuracy: 0.4790
Epoch 3/10
391/391 [==============================] - 7s 18ms/step - loss: 1.3901 - accuracy: 0.5065 - val_loss: 1.4458 - val_accuracy: 0.4912
Epoch 4/10
391/391 [==============================] - 7s 18ms/step - loss: 1.3050 - accuracy: 0.5411 - val_loss: 1.4404 - val_accuracy: 0.4952
Epoch 5/10
391/391 [==============================] - 7s 18ms/step - loss: 1.2251 - accuracy: 0.5674 - val_loss: 1.4521 - val_accuracy: 0.5019
Epoch 6/10
391/391 [==============================] - 7s 18ms/step - loss: 1.1545 - accuracy: 0.5927 - val_loss: 1.4559 - val_accuracy: 0.5033
Epoch 7/10
391/391 [==============================] - 7s 19ms/step - loss: 1.0909 - accuracy: 0.6150 - val_loss: 1.4788 - val_accuracy: 0.5067

In [14]:
model.evaluate(db_test)

model.save_weights('ckpt/weights.ckpt')

79/79 [==============================] - 1s 11ms/step - loss: 1.6406 - accuracy: 0.4902


[1.6406257605250878, 0.4902]

In [14]:
# 建立新的模型
model2 = MyModel()

model2.compile(optimizer=tf.optimizers.Adam(1e-3), 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy']
             )

In [49]:
# 模型未加载权重 直接上测试集 由下图可以看出准确率几乎为0
model2.evaluate(db_test)

79/79 [==============================] - 1s 15ms/step - loss: 2.3268 - accuracy: 0.0791


[2.3268474355528626, 0.0814]

In [51]:
# 下图的准确率 比上面不加载权重的的结果要好很多
model2.load_weights('ckpt/weights.ckpt')
model2.evaluate(db_test)

79/79 [==============================] - 1s 11ms/step - loss: 1.5239 - accuracy: 0.4613


[1.5239310581472856, 0.4613]